# Reservoirs tables - EFAS
***

**Author:** Chus Casado Rodríguez<br>
**Date:** 17-10-2024<br>

**Introduction:**<br>

* [x] 100-year return period of inflow
* [x] average inflow
* [ ] environmental flow?
* [x] storage capacity
* [x] catchment area
* [ ] reservoir area: it would be necessary if we want to model the reservoir surface area
* [ ] elevation crest: it would be necessary if we want to model the reservoir level
* [ ] dam height: it would be necessary if we want to model the reservoir level

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from scipy.stats import linregress
import cartopy.crs as ccrs
import cartopy.feature as cf

import sys
sys.path.append('E:/casadje/GitHub/lisflood-utilities/src/')
from lisfloodutilities.thresholds import compute_thresholds_gumbel

In [2]:
def compare_attributes(df, thr, vmin, vmax, **kwargs):
    """Pair plot comparing the attribute values in different data sources
    
    Parameters:
    -----------
    df: pandas.DataFrame
        Values of an attributes in different data sources (columns)
    thr: float
        Minimum value of the attribute used in the selection
    vmin: float
        Minimum attribute value to be plotted
    vmax: float
        Maximum attribute value to be plotted
    """
    
    figsize = kwargs.get('figsize', (4, 4))
    
    cols = df.columns
    ncols = len(cols) - 1
    
    fig, ax = plt.subplots(ncols=ncols, nrows=ncols, figsize=(ncols * figsize[0], ncols * figsize[1]), sharex=True, sharey=True)

    for j, colx in enumerate(cols[:-1]):
        for i, coly in enumerate(cols[1:]):
            if j > i:
                ax[i, j].axis('off')
                continue
            ax[i, j].plot([vmin, vmax], [vmin, vmax], c='k', lw=.5, zorder=0)
            ax[i, j].vlines(thr, vmin, thr, color='k', ls='--', lw=.5, zorder=0)
            ax[i, j].hlines(thr, vmin, thr, color='k', ls='--', lw=.5, zorder=0)
            ax[i, j].scatter(df[colx], df[coly], s=10, alpha=.5)
            ax[i, j].set_xscale('log')
            ax[i, j].set_yscale('log')
            if j == 0:
                ax[i, j].set_ylabel(coly)
            if i == 2:
                ax[i, j].set_xlabel(colx)

            ax[i, j].set(
                xlim=(vmin, vmax),
                ylim=(vmin, vmax),
            );
    
    if 'title' in kwargs:
        ax[0, 1].set_title(kwargs['title']);

## Configuration

In [3]:
# paths
PATH_EFAS5 = Path('Z:/nahaUsers/casadje/EFASv5')
PATH_DATASETS = Path('Z:/nahaUsers/casadje/datasets/')
PATH_EFAS6 = PATH_DATASETS / 'reservoirs' / 'EFASv6' #Path('../results/reservoirs/selection/EFAS/')

# minimum storage capacity included in EFAS
MIN_VOLUME = 10 # hm³
MIN_AREA = 50 # km²
MIN_DOR = 0.08 # 

## Data

### EFAS6
I load a shapefile with the manual selection of reservoirs to be included in EFASv6, which have already been located in the LISFLOOD grid.

In [13]:
# import shapefile
# reservoirs = gpd.read_file('Z:/H07_Global/GloFAS/data/reservoir and lakes EFAS Glofas 2024/reservoirEfasV6.shp')
efas6 = gpd.read_file(PATH_EFAS6 / 'efas6_reservoirs.shp')
efas6.GRanD_id = efas6.GRanD_id.astype('Int64')
efas6.Hylak_id = efas6.Hylak_id.astype('Int64')
efas6.ResID = efas6.ResID.astype('Int64')

# assing ID in descending order of storage capacity
efas6.sort_values('CAP_MCM', ascending=False, inplace=True)
efas6.reset_index(drop=True, inplace=True)
efas6.index.name = 'ID'
efas6.drop('ID', axis=1, errors='ignore', inplace=True)
# efas6['ID'] = efas6.index

### EFAS5

In [9]:
efas5 = gpd.read_file(PATH_EFAS6 / 'EFAS5' / 'efas5_reservoirs.shp')#.set_index('ResID')

# assign ID
map_efas_ID = {ResID: ID for ID, ResID in zip(efas6.index, efas6.ResID) if not pd.isna(ResID)}
efas5['ID'] = efas5.ResID.map(map_efas_ID).astype('Int64')
efas5.dropna(subset=['ID'], inplace=True)
efas5.set_index('ID', drop=False, inplace=True)

print(efas5.shape)

(1393, 34)


### GRanD

In [22]:
grand = gpd.read_file(PATH_EFAS6 / 'GRanD' / 'grand_dams.shp')

# connection between GRanD_id and ID
map_grand_ID = {grand_id: ID for ID, grand_id in zip(efas6.index, efas6.GRanD_id) if not pd.isna(grand_id)}
grand['ID'] = grand.GRAND_ID.map(map_grand_ID).astype('Int64')
grand.rename(columns={'GRAND_ID': 'GRanD_id'}, inplace=True)
grand.set_index('GRanD_id', drop=True, inplace=True)

print(grand.shape)

(1575, 62)


### ICOLD

In [67]:
icold = gpd.read_file(PATH_EFAS6 / 'ICOLD' / 'icold.shp')#.set_index('Int_Code')

# connection between ICOLD and ID
map_icold_ID = {icold_id: ID for ID, icold_id in zip(efas6.index, efas6.ICOLD_id) if not pd.isna(icold_id)}
icold['ID'] = icold.Int_Code.map(map_icold_ID).astype('Int64')
icold.set_index('Int_Code', drop=True, inplace=True)

print(icold.shape)

(7887, 46)


### HydroLakes

In [66]:
hylak = gpd.read_file(PATH_EFAS6 / 'HydroLakes' / 'hydrolakes_dams.shp')#.set_index('Hylak_id')

# connection between ICOLD and ID
map_hylak_ID = {hylak_id: ID for ID, hylak_id in zip(efas6.index, efas6.Hylak_id) if not pd.isna(hylak_id)}
hylak['ID'] = hylak.Hylak_id.map(map_hylak_ID).astype('Int64')
hylak.set_index('Hylak_id', drop=True, inplace=True)

print(hylak.shape)

(1415, 307)


## Attributes
### Capacity

In [ ]:
capacity = pd.DataFrame(index=efas6.index, columns=['EFAS5', 'GRAND', 'ICOLD', 'HYLAK', 'EFAS6', 'SOURCE'], dtype=float)
capacity.SOURCE = capacity.SOURCE.astype(str)

# EFAS5
capacity.loc[efas5.index, 'EFAS5'] = efas5.tstor

# GRanD
# capacity.loc[efas6.index, 'GRAND'] = efas6.CAP_MCM
mask = grand.ID.notnull()
capacity.loc[grand.loc[mask, 'ID'], 'GRAND'] = grand.loc[mask, 'CAP_MCM'].values

# ICOLD
# capacity.loc[efas5.index, 'ICOLD'] = efas5.ICOLD_VOL
mask = icold.ID.notnull()
capacity.loc[icold.loc[mask, 'ID'], 'ICOLD'] = icold.loc[mask, 'Res_Vol'].values

# HydroLakes
mask = hylak.ID.notnull()
capacity.loc[hylak.loc[mask, 'ID'], 'HYLAK'] = hylak.loc[mask, 'Vol_res'].values

#### Comparison

In [ ]:
compare_attributes(
    capacity[['EFAS5', 'GRAND', 'ICOLD', 'HYLAK']],
    thr=MIN_VOLUME,
    vmin=1e-1,
    vmax=1e6,
    title='storage (hm3)'
)

In [ ]:
# error = 2
# mask_error = (capacity.GRAND < capacity.EFAS5 / error) | (capacity.GRAND > error * capacity.EFAS5)
# print(mask_error.sum())
# capacity[mask_error]

#### Select values

I have manually checked the large discrepancies between EFAS5 and GRanD. In case of discrepancy, I compare with ICOLD and HydroLakes. It seems like GRanD is a more reliable source, so I will use the value from GRanD whenever possible, if not the value from EFAS5.

In [ ]:
# select value from GRAND, EFAS5, HYLAK or ICOLD in that order
for source in ['GRAND', 'EFAS5', 'HYLAK', 'ICOLD']:
    missing = capacity.EFAS6.isnull()
    mask = capacity[source].notnull()
    capacity.loc[missing & mask, 'EFAS6'] = capacity.loc[missing & mask, source]
    capacity.loc[missing & mask, 'SOURCE'] = source

# add values to EFAS6
efas6.loc[capacity.index, ['VOL_MCM', 'VOL_SRC']] = capacity[['EFAS6', 'SOURCE']].values

# # storage of reservoirs coming from EFAS5
# efas6[efas6.VOL_SRC == 'EFAS5']['VOL_MCM']

print('{0} reservoirs do not comply with the minimum storage of {1} hm3'.format((efas6.VOL_MCM < MIN_VOLUME).sum(),
                                                                                MIN_VOLUME))

### Catchment area

In [ ]:
area = pd.DataFrame(index=efas6.index, columns=['EFAS5', 'GRAND', 'ICOLD', 'HYLAK', 'EFAS6', 'SOURCE'], dtype=float)
area.SOURCE = area.SOURCE.astype(str)

# EFAS5
area.loc[efas5.index, 'EFAS5'] = efas5['area']

# # GRanD
# area.loc[efas6.index, 'GRAND'] = efas6.CAP_MCM
mask = grand.ID.notnull()
area.loc[grand.loc[mask, 'ID'], 'GRAND'] = grand.loc[mask, 'CATCH_SKM'].values

# ICOLD
area.loc[efas5.index, 'ICOLD'] = efas5.ICOLD_AREA
mask = icold.ID.notnull()
area.loc[icold.loc[mask, 'ID'], 'ICOLD'] = icold.loc[mask, 'Catch_Area'].values

# HydroLakes
mask = hylak.ID.notnull()
area.loc[hylak.loc[mask, 'ID'], 'HYLAK'] = hylak.loc[mask, 'Wshd_area'].values

#### Comparison

In [ ]:
compare_attributes(
    area[['EFAS5', 'GRAND', 'ICOLD', 'HYLAK']],
    thr=MIN_AREA,
    vmin=1,
    vmax=1e7,
    title='catchment (km²)'
)

#### Select values

Following the procedure applied for the storage capacity, I will use the GRanD value when available.

In [ ]:
# select value from GRAND, EFAS5, HYLAK or ICOLD in that order
for source in ['GRAND', 'EFAS5', 'HYLAK', 'ICOLD']:
    missing = area.EFAS6.isnull()
    mask = area[source].notnull()
    area.loc[missing & mask, 'EFAS6'] = area.loc[missing & mask, source]
    area.loc[missing & mask, 'SOURCE'] = source

# add values to EFAS6
efas6.loc[area.index, ['CATCH_SKM', 'CATCH_SRC']] = area[['EFAS6', 'SOURCE']].values

# # storage of reservoirs coming from EFAS5
# efas6[efas6.VOL_SRC == 'EFAS5']['VOL_MCM']

print('{0} reservoirs do not comply with the minimum catchment area of {1} km²'.format((efas6.CATCH_SKM < MIN_AREA).sum(),
                                                                                       MIN_AREA))

## `ncextract`

I run the tool `ncextract` on the EFASv5 long-term run simulation with natural flow, i.e, no reservoirs and no water abstraction. The discharge files resulting from this simulation are in the HPC: `/BGFS/DISASTER/casadje/EFASv5/natural_flow/`.

The discharge time series extracted for the reservoir locations are proxies of the reservoir inflow. From these time series we need to estimate the 100-year return period and normal inflow.

In [ ]:
# export CSV needed to run `ncextract`
points = efas6[['LisfloodX', 'LisfloodY']]
points.columns = ['lon', 'lat']
points.to_csv(PATH_EFAS6 / 'ncextract' / 'reservoirs_efas6_new.csv')

## `thresholds`

In [ ]:
PATH_OUT = PATH_EFAS6 / 'tables'
PATH_OUT.mkdir(parents=True, exist_ok=True)

In [ ]:
# load discharge timeseries
dis_nat1 = xr.open_dataset(PATH_EFAS6 / 'ncextract' / 'inflow_natural_ltr.nc')['dis']
dis_nat1.close()

In [ ]:
# load discharge timeseries
dis_nat2 = xr.open_dataset(PATH_EFAS6 / 'ncextract' / 'natural_flow_prerunLZAVIN.nc')['dis']
dis_nat2.close()

In [ ]:
# load discharge timeseries
dis_ltr = xr.open_dataset(PATH_EFAS6 / 'ncextract' / 'inflow_ltr.nc')['dis']
dis_ltr.close()
# daily mean
dis_ltr = dis_ltr.resample(time='1D').mean()

In [ ]:
ID = 4

lw = .8
for ID in tqdm(dis_nat2.ID.data):
    fig, ax = plt.subplots(figsize=(12, 3))
    dis_ltr.sel(ID=ID).plot(lw=lw, ax=ax, label='altered')
    dis_nat1.sel(ID=ID).sel(time=slice('1993-01-02', None)).plot(lw=lw, ax=ax, label='natural')
    dis_nat2.sel(ID=ID).sel(time=slice('1993-01-02', None)).plot(lw=lw, ax=ax, label='natural lzavin')
    ax.legend(frameon=False)
    ax.set_title(ID)
    # ax.set(xlim=(dis_nat.time.min().item(), dis_nat.time.max().item()))
    plt.savefig(PATH_EFAS6 / 'ncextract' / 'plots' / 'natural_vs_lzavin_vs_regular' / f'{ID}.jpg', dpi=300, bbox_inches='tight')
    plt.close(fig)

In [ ]:
avg_dis_nat1 = dis_nat1.sel(time=slice('1993-01-01', None)).groupby('time.year').mean().to_pandas()

linreg = pd.DataFrame(index=avg_dis_nat1.columns, columns=['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])
for ID in avg_dis_nat1.columns:
    linreg.loc[ID,:] = linregress(avg_dis_nat1.index, avg_dis_nat1[ID] / avg_dis_nat1[ID].mean())

IDs1 = linreg[(linreg.pvalue < 0.05) & (linreg.slope < -0.05)].index.tolist()
print(len(IDs))

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cf.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', facecolor='lightgray'), alpha=.5, zorder=0)
ax.axis('off')
efas6.loc[IDs1].plot(markersize=5, ax=ax);
# plt.savefig(PATH_EFAS6 / 'ncextract' / 'plots' / 'natural' / 'map.jpg', dpi=300, bbox_inches='tight')
# plt.close(fig);

In [ ]:
avg_dis_nat2 = dis_nat2.sel(time=slice('1993-01-01', None)).groupby('time.year').mean().to_pandas()

linreg = pd.DataFrame(index=avg_dis_nat2.columns, columns=['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])
for ID in avg_dis_nat2.columns:
    linreg.loc[ID,:] = linregress(avg_dis_nat2.index, avg_dis_nat2[ID] / avg_dis_nat2[ID].mean())

IDs2 = linreg[(linreg.pvalue < 0.05) & (linreg.slope < -0.05)].index.tolist()
print(len(IDs2))

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cf.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', facecolor='lightgray'), alpha=.5, zorder=0)
ax.axis('off')
efas6.loc[IDs2].plot(markersize=5, ax=ax);
# plt.savefig(PATH_EFAS6 / 'ncextract' / 'plots' / 'natural' / 'map.jpg', dpi=300, bbox_inches='tight')
# plt.close(fig);

In [ ]:
# efas6.drop('ID', axis=1, inplace=True)
# efas6.loc[IDs].to_file(PATH_EFAS6 / 'ncextract' / 'points_to_test.shp')

In [ ]:
avg_dis_nat2.index = pd.to_datetime([f'{year}-07-01' for year in avg_dis_nat2.index])
lw = .8
for ID in tqdm(IDs2):
    fig, ax = plt.subplots(figsize=(12, 3))
    dis_ltr.sel(ID=ID).plot(lw=lw, ax=ax, label='altered')
    dis_nat1.sel(ID=ID).sel(time=slice('1993-01-02', None)).plot(lw=lw, ax=ax, label='natural')
    dis_nat2.sel(ID=ID).sel(time=slice('1993-01-02', None)).plot(lw=lw, ax=ax, label='natural')
    avg_dis_nat2[ID].plot(c='C1', ls='--', lw=lw*.8, ax=ax, label='natural_trend')
    ax.legend(frameon=False)
    ax.set_title(ID)
    # ax.set(xlim=(dis_nat.time.min().item(), dis_nat.time.max().item()))
    # plt.savefig(PATH_EFAS6 / 'ncextract' / 'plots' / 'natural_vs_regular' / f'{ID}.jpg', dpi=300, bbox_inches='tight')
    # plt.close(fig)

In [ ]:
dis_nat.sel(ID=1515, time=slice('1993-01-01', None)).plot()

In [ ]:
avg_dis_nat.plot()

In [ ]:
linreg.pvalue

In [ ]:
# # load discharge timeseries
# dis_efas5 = xr.open_dataset(PATH_EFAS6 / 'ncextract' / 'dis_EFAS5_extract.nc')

In [ ]:
aux = xr.open_dataset(PATH_EFAS6 / 'ncextract' / 'inflow_ltr_points.nc')['dis']
aux.close()
aux

In [ ]:
PATH_EFAS6

In [ ]:
xmin, xmax, ymin, ymax = 33.5, 45.5, 31.5, 38
start, end = '1993-01-02', None # '1997-01-01'
for ID in aux.ID.data:
    lon, lat = dis.sel(ID=ID).lon.item(), dis.sel(ID=ID).lat.item()
    if (xmin < lon) & (lon < xmax) & (ymin < lat) & (lat < ymax):
        fig, ax = plt.subplots(figsize=(12, 3))
        aux.sel(ID=ID).sel(time=slice(start, end)).plot(ax=ax, label='altered');
        dis.sel(ID=ID).sel(time=slice(start, end)).plot(ax=ax, label='natural')
        ax.set(ylim=(-10, None), title=ID)
        ax.legend(frameon=False);
        # plt.savefig(PATH_EFAS6 / 'ncextract' / 'plots' / f'{ID}.jpg', dpi=300, bbox_inches='tight')

In [ ]:
efas6.shape

In [ ]:
IDs

In [ ]:
import random

IDs = [random.randint(1, 1544) for i in range(20)]
for ID in IDs:
    dis.sel(ID=ID).sel(time=slice('1993-01-02', None)).plot(figsize=(12, 3))
    plt.title(ID);

In [ ]:
dis_ltr.time.data[-1]

In [ ]:
dis_ltr.sel(ID=12).lon.item(), dis_ltr.sel(ID=12).lat.item()

In [ ]:
dis.sel(ID=12).lon.item(), dis.sel(ID=12).lat.item()

In [ ]:
dis.sel(ID=14).sel(time=slice('1993-01-02', None)).plot(figsize=(12, 3));

In [ ]:
efas6.iloc[[12, 100, 4]]

In [ ]:
dis.isel(ID=15).sel(time=slice('1993-01-02', None)).plot(figsize=(12, 3));

In [ ]:
dis.isel(ID=18).plot(figsize=(12, 3));

In [ ]:
# remove first year to avoid initialization errors
dis = dis.sel(time=slice('1993-01-02', None))

# keep selected reservoirs
dis = dis.sel(ID=efas6.index.tolist())

# # remove IDs with all NaN
# print('IDs with all NaN: {0}'.format(*dis.ID[dis.isnull().all('time')].data))
# dis = dis.dropna(dim='ID', how='all')

### Mean inflow

In [ ]:
# mean inflow
efas6['AVG_INFLOW'] = dis.mean('time').to_pandas()
efas6['AVG_INFLOW'].to_csv(PATH_OUT / 'inflow_mean.txt', header=None, sep=' ', float_format='%.1f')

### Degree of regulation

In [ ]:
efas6['DOR'] = efas6.VOL_MCM * 1e6 / (efas6.AVG_INFLOW  * 24 * 3600 * 365) 

print('{0} reservoirs do not comply with the minimum degree of regulation {1}'.format((efas6.DOR < MIN_DOR).sum(),
                                                                                       MIN_DOR))

### 100-year return period

In [ ]:
# annual maxima
dis_max = dis.groupby('time.year').max(dim='time')#.rename({'year': 'time'})

# inflow return periods
return_periods = [100]
thresholds = compute_thresholds_gumbel(dis_max, return_periods=return_periods, dim='year')

In [ ]:
# export discharge values
for rp in return_periods:
    var = f'INFLOW_{rp}'
    efas6[var] = thresholds[f'rp_{rp}'].to_pandas()
    efas6[var].to_csv(PATH_OUT / f'{var.lower()}.txt', header=None, sep=' ', float_format='%.1f')

In [ ]:
# export parameters
for par in ['mu', 'sigma']:
    thresholds[par].to_pandas().to_csv(PATH_OUT / f'gumbel_{par}.txt', header=None, sep=' ', float_format='%.1f')

In [ ]:
# export input shapefile with added attributes
efas6.to_file(PATH_EFAS6 / 'efas6_reservoirs.shp')

***

## EFAS Annual Meeting

In [85]:
# copyt efas6 as main source
jrcbox = efas6.copy()
jrcbox.drop(['COMMENTS', 'ToAdd', 'grid_code', 'EFAS5'], axis=1, inplace=True)

In [86]:
# add data from GRanD
grand_ids = jrcbox.GRanD_id.dropna()
grand_cols = ['RES_NAME', 'DAM_NAME', 'RIVER', 'MAIN_BASIN', 'COUNTRY', 'YEAR', 'LONG_DD', 'LAT_DD']
jrcbox.loc[grand_ids.index, grand_cols] = grand.loc[grand_ids.values, grand_cols].values

In [87]:
# mask_no_grand = jrcbox.GRanD_id.isnull()
# mask_efas5 = jrcbox.ResID.notnull()

In [88]:
# add info from EFAS5
ids = jrcbox[jrcbox.GRanD_id.isnull() & jrcbox.ResID.notnull()].index
cols_efas5 = ['COUNTRY', 'DAM_NAME']
# jrcbox.loc[ids, cols_efas5] = efas5.loc[ids, cols_efas5]

In [89]:
# reorder and rename columns
cols = ['ResID', 'GRanD_id', 'Hylak_id', 'ICOLD_id', 
        'RES_NAME', 'DAM_NAME', 'RIVER', 'MAIN_BASIN', 'COUNTRY', 'YEAR',
        'LONG_DD', 'LAT_DD', 'LisfloodX', 'LisfloodY',
        'VOL_MCM', 'VOL_SRC', 'CATCH_SKM', 'CATCH_SRC', 'AVG_INFLOW', 'INFLOW_100', 'DOR',
        'geometry']
jrcbox = jrcbox[cols]
jrcbox.rename(columns={'VOL_MCM': 'CAP_MCM', 'VOL_SRC': 'CAP_SRC'}, inplace=True)
jrcbox.YEAR = jrcbox.YEAR.astype('Int64')

In [91]:
# export
path_jrcbox = Path('E:/casadje/jrcbox/Documents/EFAS6/reservoirs')
jrcbox.to_file(path_jrcbox / 'GIS' / 'efas6_reservoirs.shp')
jrcbox.drop('geometry', axis=1).to_excel(path_jrcbox / 'table' / 'efas6_reservoirs.xlsx')